In [48]:
import os
import tvm
from tvm.script import ir as I
from tvm.script import tir as T
from tvm import autotvm, auto_scheduler
from tvm.autotvm.tuner import XGBTuner, GATuner, RandomTuner, GridSearchTuner
from tvm import meta_schedule as ms
from tvm.ir import IRModule
from tvm import relax
from tvm import rpc
from tvm.contrib import utils, ndk
x_shape = 11008
w_w_x = 1376
w_s_x = 344
w_y = 4096
func_name = "main"
@I.ir_module
class ModuleSrc:
    @T.prim_func(private=False)
    def main(lv575: T.Buffer((T.int64(1376), T.int64(4096)), "uint32"), lv576: T.Buffer((T.int64(344), T.int64(4096)), "float16"), lv574: T.Buffer((T.int64(1), T.int64(1), T.int64(11008)), "float16"), lv570: T.Buffer((T.int64(1), T.int64(1), T.int64(4096)), "float16"), p_output0_intermediate: T.Buffer((T.int64(1), T.int64(1), T.int64(4096)), "float16")):
        T.func_attr({"global_symbol": "main", "tir.noalias": T.bool(True)})
        # with T.block("root"):
        p_output0_intermediate_1 = T.alloc_buffer((T.int64(11008), T.int64(4096)), "float16")
        var_matmul_intermediate = T.alloc_buffer((T.int64(1), T.int64(1), T.int64(4096)), "float16")
        for i, j in T.grid(T.int64(11008), T.int64(4096)):
            with T.block("decode"):
                v_i, v_j = T.axis.remap("SS", [i, j])
                T.reads(lv575[v_i // T.int64(8), v_j], lv576[v_i // T.int64(32), v_j])
                T.writes(p_output0_intermediate_1[v_i, v_j])
                p_output0_intermediate_1[v_i, v_j] = (T.Cast("float16", T.bitwise_and(T.shift_right(lv575[v_i // T.int64(8), v_j], T.Cast("uint32", v_i % T.int64(8)) * T.uint32(4)), T.uint32(15))) - T.float16(7)) * lv576[v_i // T.int64(32), v_j]
        for i0, i1, i2, k in T.grid(T.int64(1), T.int64(1), T.int64(4096), T.int64(11008)):
            with T.block("matmul"):
                v_i0, v_i1, v_i2, v_k = T.axis.remap("SSSR", [i0, i1, i2, k])
                T.reads(lv574[v_i0, v_i1, v_k], p_output0_intermediate_1[v_k, v_i2])
                T.writes(var_matmul_intermediate[v_i0, v_i1, v_i2])
                with T.init():
                    var_matmul_intermediate[v_i0, v_i1, v_i2] = T.float16(0)
                var_matmul_intermediate[v_i0, v_i1, v_i2] = var_matmul_intermediate[v_i0, v_i1, v_i2] + lv574[v_i0, v_i1, v_k] * p_output0_intermediate_1[v_k, v_i2]
        for ax0, ax1, ax2 in T.grid(T.int64(1), T.int64(1), T.int64(4096)):
            with T.block("T_add"):
                v_ax0, v_ax1, v_ax2 = T.axis.remap("SSS", [ax0, ax1, ax2])
                T.reads(lv570[v_ax0, v_ax1, v_ax2], var_matmul_intermediate[v_ax0, v_ax1, v_ax2])
                T.writes(p_output0_intermediate[v_ax0, v_ax1, v_ax2])
                p_output0_intermediate[v_ax0, v_ax1, v_ax2] = lv570[v_ax0, v_ax1, v_ax2] + var_matmul_intermediate[v_ax0, v_ax1, v_ax2]

@I.ir_module
class ModuleToManual:
    @T.prim_func(private=False)
    def main(lv575: T.Buffer((T.int64(1376), T.int64(4096)), "uint32"), lv576: T.Buffer((T.int64(344), T.int64(4096)), "float16"), lv574: T.Buffer((T.int64(1), T.int64(1), T.int64(11008)), "float16"), lv570: T.Buffer((T.int64(1), T.int64(1), T.int64(4096)), "float16"), p_output0_intermediate: T.Buffer((T.int64(1), T.int64(1), T.int64(4096)), "float16")):
        T.func_attr({"global_symbol": "main", "tir.noalias": T.bool(True)})
        # with T.block("root"):
        p_output0_intermediate_1 = T.alloc_buffer((T.int64(11008), T.int64(4096)), "float16")
        var_matmul_intermediate = T.alloc_buffer((T.int64(1), T.int64(1), T.int64(4096)), "float16")
        for i, j in T.grid(T.int64(11008), T.int64(4096)):
            with T.block("decode"):
                v_i, v_j = T.axis.remap("SS", [i, j])
                T.reads(lv575[v_i // T.int64(8), v_j], lv576[v_i // T.int64(32), v_j])
                T.writes(p_output0_intermediate_1[v_i, v_j])
                p_output0_intermediate_1[v_i, v_j] = (T.Cast("float16", T.bitwise_and(T.shift_right(lv575[v_i // T.int64(8), v_j], T.Cast("uint32", v_i % T.int64(8)) * T.uint32(4)), T.uint32(15))) - T.float16(7)) * lv576[v_i // T.int64(32), v_j]
        for i0, i1, i2, k in T.grid(T.int64(1), T.int64(1), T.int64(4096), T.int64(11008)):
            with T.block("matmul"):
                v_i0, v_i1, v_i2, v_k = T.axis.remap("SSSR", [i0, i1, i2, k])
                T.reads(lv574[v_i0, v_i1, v_k], p_output0_intermediate_1[v_k, v_i2])
                T.writes(var_matmul_intermediate[v_i0, v_i1, v_i2])
                with T.init():
                    var_matmul_intermediate[v_i0, v_i1, v_i2] = T.float16(0)
                var_matmul_intermediate[v_i0, v_i1, v_i2] = var_matmul_intermediate[v_i0, v_i1, v_i2] + lv574[v_i0, v_i1, v_k] * p_output0_intermediate_1[v_k, v_i2]
        for ax0, ax1, ax2 in T.grid(T.int64(1), T.int64(1), T.int64(4096)):
            with T.block("T_add"):
                v_ax0, v_ax1, v_ax2 = T.axis.remap("SSS", [ax0, ax1, ax2])
                T.reads(lv570[v_ax0, v_ax1, v_ax2], var_matmul_intermediate[v_ax0, v_ax1, v_ax2])
                T.writes(p_output0_intermediate[v_ax0, v_ax1, v_ax2])
                p_output0_intermediate[v_ax0, v_ax1, v_ax2] = lv570[v_ax0, v_ax1, v_ax2] + var_matmul_intermediate[v_ax0, v_ax1, v_ax2]

In [49]:
## ref to mlc-llm/dispatch/dispatch_tir_operator_adreno.py
# 最优配置：0.711104ms
## vf	vi	tx	ty	bx
## 4	2	128	8	8
def sch_fused_decode6_fused_matmul7_add1(func):
    sch = tvm.tir.Schedule(func)
    b0 = sch.get_block(name="decode", func_name="main")
    b1 = sch.get_block(name="matmul", func_name="main")
    # 1 1 4096 11008
    l2, l3, l4, l5 = sch.get_loops(block=b1)
    # l6 = 4096
    l6 = sch.fuse(l2, l3, l4, preserve_unit_iters=True)
    v7, v8, v9 = sch.sample_perfect_tile(
        # loop=l6, n=3, max_innermost_factor=4, decision=[8, 256, 2]
        loop=l6, n=3, max_innermost_factor=4, decision=[8, 128, 4]
    )
    # 8  128   4
    l10, l11, l12 = sch.split(loop=l6, factors=[v7, v8, v9], preserve_unit_iters=True)
    v13, v14, v15 = sch.sample_perfect_tile(
        loop=l5, n=3, max_innermost_factor=8, decision=[344, 4, 8]
    )
    l16, l17, l18 = sch.split(
        loop=l5, factors=[v13, v14, v15], preserve_unit_iters=True
    )
    ## 搜索记录:
    #       2, 172  0.939960ms
    #       4, 86   0.757256ms
    #       8, 43   716400ms
    # v161, v162 = sch.sample_perfect_tile(loop=l16, n=2, max_innermost_factor=172, decision=[4, 86])
    l161, _ = sch.split(l16, [8, 43], preserve_unit_iters=True)
    # 将reduce操作按照l161做拆分，以便并行化. 其过程是: 
    #   1. 申请 [l161, 1, 1, 4096]个xxx_rf内存
    #   2. 将计算拆为l161个chunks分别计算
    #   3. 按照l161维度reduce, 得到最终的输出
    sch.rfactor(l161, 0)
    ## 此时b1还是block('matmul'), 但是计算内容变为rf的合并操作(即: reduce = sum(sub_reduces))
    ## 此时拆分的reduce计算变为 block('matmul_rf')
    b2 = sch.get_block(name="matmul_rf", func_name = "main")
    # 调用rfactor后, 计算部分转移到了 block('matmul_rf'), 所以得重新获取loops
    # s = spatial,  r = reduce
    lb2s1, lb2s2, lb2s3, lb2r1, lb2r2, lb2r3, lb2r4 = sch.get_loops(block=b2)
    # 8, 128, 8, 43, 4, 8, 4
    sch.reorder(lb2s1, lb2s2, lb2r1, lb2r2, lb2r3, lb2r4, lb2s3)
    sch.bind(loop=lb2s1, thread_axis="blockIdx.x")
    sch.bind(loop=lb2s2, thread_axis="threadIdx.x")
    sch.bind(loop=lb2r1, thread_axis="threadIdx.y")

    sch.compute_inline(block=b0)
    b21 = sch.cache_write(block = b2, write_buffer_index=0, storage_scope="local")
    sch.reverse_compute_at(block = b21, loop=lb2r1, preserve_unit_loops=True, index=-1)
    lb21, lb22, lb23, lb24, lb25, lb26, lb27 = sch.get_loops(block=b2)
    sch.vectorize(lb27)
    lb211, lb212, lb213, lb214, lb215,lb216, lb217 = sch.get_loops(block=b21)
    sch.vectorize(lb217)

    # 权重W uint32: [1376, 4096]
    b80 = sch.cache_read(block=b2, read_buffer_index=1, storage_scope="local")
    sch.compute_at(block=b80, loop=lb2r3, preserve_unit_loops=True, index=-1)
    l801, l802, l803, l804, l805, l806, l807 = sch.get_loops(block=b80)
    sch.vectorize(l807)

    # 权重Scale fp16: [344, 4096]
    b81 = sch.cache_read(block=b2, read_buffer_index=2, storage_scope="local")
    sch.compute_at(block=b81, loop=lb2r2, preserve_unit_loops=True, index=-1)
    l811, l812, l813, l814, l815, l816 = sch.get_loops(block=b81)
    sch.vectorize(l816)

    # 输入X: lv574_shared, shape[1,1,11008] -> [8, 128, 8, 1, 1, 11008/(8*128*8)] (无法整除)-> [8, 128, 8, 1, 1, 11008]
    b20 = sch.cache_read(block=b2, read_buffer_index=0, storage_scope="shared")
    sch.compute_at(block=b20, loop=lb2r1, preserve_unit_loops=True, index=-1)
    lb20l1, lb20l2, lb20l3, lb20l4, lb20l5, lb20l6 = sch.get_loops(block=b20)
    l34, l35, l36, l37 = sch.split(
        # loop=l33, factors=[None, 256, 8], preserve_unit_iters=True
        ### 注意这里的数字与最开始设置保持一致
        # loop=lb20l6, factors=[None, 256, 4, 4], preserve_unit_iters=True
        loop=lb20l6, factors=[None, 128, 8, 2], preserve_unit_iters=True
    )
    # 这里 l37 = 4, 11008 / (threadIdx.x * threadIdx.y * 4) 无法整除, 所以只有一部分线程可以执行到shared_memory构建
    # 实际观测 sch.vectorize(loop=l37) 并没有生效，不知道是否因为不是所有线程都能执行, 所以取消了vload4
    sch.vectorize(loop=l37)
    sch.bind(loop=l35, thread_axis="threadIdx.x")
    sch.bind(loop=l36, thread_axis="threadIdx.y")
    v21 = sch.sample_categorical(
        candidates=[1, 2, 4, 8], probs=[0.25, 0.25, 0.25, 0.25], decision=2
    )
    sch.annotate(
        block_or_loop=b20, ann_key="meta_schedule.cooperative_fetch", ann_val=v21
    )
    sch.vectorize(loop=l12)
    b27 = sch.decompose_reduction(block=b2, loop=lb2r1)
    sch.enter_postproc()
    sch.unannotate(block_or_loop=b20, ann_key="meta_schedule.cooperative_fetch")

    #######################################################################################
    ###################以下为kenenl1: 执行 sub_reduce结果合并 & bias操作#######################
    #######################################################################################
    # 计算的中间结果: var_matmul_intermediate_rf, shape[8, 1,1,4096]
    #  8   128   4
    # l10, l11, l12
    b19 = sch.cache_write(block=b1, write_buffer_index=0, storage_scope="local")
    l41,l42,l43,l44 = sch.get_loops(block=b1)
    l411 = sch.fuse(l41,l42, l43)
    ## 调整记录:
    #       4, 256, 4 0.717400ms
    #       8, 128, 4 0.717368ms
    #       16, 64, 4 0.717368ms
    #       32, 32, 4 0.716808ms
    #       64, 16, 4 0.718056ms

    #       4, 128, 8 0.719936ms
    #       8, 64,  8 0.719024ms
    #       16, 32, 8 0.716400ms best
    #       32, 16, 8 0.717544ms

    #       4, 64, 16 0.722680ms
    #       8, 32, 16 0.719608ms
    #       16, 16, 16 0.718744ms
    #       32, 8, 16 0.720376ms
    l411_1, l411_2, l411_3 = sch.split(l411, [16, 32, 8], preserve_unit_iters=True)
    sch.reverse_compute_at(block=b19, loop=l411_2, preserve_unit_loops=True, index=-1)
    b28 = sch.get_block(name="T_add", func_name="main")
    sch.reverse_compute_inline(block=b28)
    # b19 对应block('matmul')的write部分，其实对应的是 block('T_add') write
    # 8 128 1 1 4
    l22, l23, l24, l25, l26 = sch.get_loops(block=b19)
    sch.vectorize(loop=l26)
    print(sch.get(l22))
    # 8 128 4 8
    l41,l42,l43,l44 = sch.get_loops(block=b1)
    sch.vectorize(loop=l43)
    print("================================================================")
    print(sch.get(l41))
    sch.bind(loop=l41, thread_axis="blockIdx.x")
    sch.bind(loop=l42, thread_axis="threadIdx.x")
    return sch.mod["main"].with_attr("tir.is_scheduled", 1)


sch_manual = tvm.tir.Schedule(ModuleToManual)
sch_manual.mod['main'] = sch_fused_decode6_fused_matmul7_add1(sch_manual.mod[func_name])
# print("\n\n=================final mod===============================")
# print(sch_manual.mod.script())
print("\n\n=================kernel source===============================")
rt_mod = tvm.build(sch_manual.mod, target="opencl")
print(rt_mod.imported_modules[0].get_source())

for i0_i1_i2_fused_0_i0_i1_i2_fused_1_i0_i1_i2_fused_2_fused_0, i0_i1_i2_fused_0_i0_i1_i2_fused_1_i0_i1_i2_fused_2_fused_1 in T.grid(T.int64(16), T.int64(32)):
    var_matmul_intermediate_local = T.Buffer((T.int64(1), T.int64(1), T.int64(4096)), "float16", scope="local")
    for i0_i1_i2_fused_0_i0_i1_i2_fused_1_i0_i1_i2_fused_2_fused_2, k_0_0 in T.grid(T.int64(8), T.int64(8)):
        with T.block("matmul"):
            vk_0_0 = T.axis.reduce(T.int64(8), k_0_0)
            v_i0 = T.axis.spatial(T.int64(1), T.int64(0))
            v_i1 = T.axis.spatial(T.int64(1), T.int64(0))
            v_i2 = T.axis.spatial(T.int64(4096), i0_i1_i2_fused_0_i0_i1_i2_fused_1_i0_i1_i2_fused_2_fused_0 * T.int64(256) + i0_i1_i2_fused_0_i0_i1_i2_fused_1_i0_i1_i2_fused_2_fused_1 * T.int64(8) + i0_i1_i2_fused_0_i0_i1_i2_fused_1_i0_i1_i2_fused_2_fused_2)
            var_matmul_intermediate_rf = T.Buffer((T.int64(8), T.int64(1), T.int64(1), T.int64(4096)), "float16")
            T.reads(var_matmul_intermediate_

In [55]:
# run and compare with cuda
import numpy as np
def _detect_local_cuda():
    dev = tvm.cuda()
    if not dev.exist:
        return None
    return tvm.target.Target(
        {
            "kind": "cuda",
            "max_shared_memory_per_block": dev.max_shared_memory_per_block,
            "max_threads_per_block": dev.max_threads_per_block,
            "thread_warp_size": dev.warp_size,
            "registers_per_block": 65536,
            "arch": "sm_" + tvm.cuda().compute_version.replace(".", ""),
        }
    )
# target = tvm.target.Target("cuda", host="llvm")
target = _detect_local_cuda()

print(target)
# 定义计算任务
dev = tvm.cuda(0)

num_flop = 1228406784
W_w_np = np.random.uniform(size=(w_w_x, w_y)).astype("uint32")
W_s_np = np.random.uniform(size=(w_s_x, w_y)).astype("float16")
B_np = np.random.uniform(size=(1, 1, w_y)).astype("float16")
Input_np = np.random.uniform(size=(1, 1, x_shape)).astype("float16")
# W_w_np = np.ones((w_w_x, w_y), np.uint32) * 1#.astype("uint32")
# W_s_np = np.ones((w_s_x, w_y), np.float16) * 1#.astype("float16") * 2
# Input_np = np.ones((1, 1, x_shape), np.float16)#.astype("float16")
Output_nd = tvm.nd.array(np.zeros((1, 1, w_y), dtype="float16"), dev)
def numpy_caculate():
    test_cols = 10
    reduce_div_factor = 8
    per_reduce_div_shape = x_shape//reduce_div_factor
    output = np.zeros((1, 1, test_cols), dtype = np.float16)
    output_rf = np.zeros((reduce_div_factor, 1, 1, test_cols), dtype = np.float16)
    W_w_inv_np = np.transpose(W_w_np)
    W_s_inv_np = np.transpose(W_s_np)
    for i in range(test_cols):
        for rf in range(reduce_div_factor):
            for r in range(per_reduce_div_shape * rf, per_reduce_div_shape * (rf+1)):
                temp = Input_np[0][0][r] * np.float16((W_w_inv_np[i][r // 8] >> ((r % 8) * 4) & (15)) - np.float16(7.0)) * W_s_inv_np[i][r // 32]
                output_rf[rf][0][0][i] = output_rf[rf][0][0][i] + temp
            output[0][0][i] = output[0][0][i] + output_rf[rf][0][0][i]
        output[0][0][i] = output[0][0][i] + B_np[0][0][i]
    print(output)

    output = np.zeros((1, 1, test_cols), dtype = np.float16)
    output_rf = np.zeros((reduce_div_factor, 1, 1, test_cols), dtype = np.float16)
    for i in range(test_cols):
        for rf in range(reduce_div_factor):
            for r in range(per_reduce_div_shape * rf, per_reduce_div_shape * (rf+1)):
                temp = Input_np[0][0][r] * np.float16((W_w_np[r // 8][i] >> ((r % 8) * 4) & (15)) - np.float16(7.0)) * W_s_np[r // 32][i]
                output_rf[rf][0][0][i] = output_rf[rf][0][0][i] + temp
            output[0][0][i] = output[0][0][i] + output_rf[rf][0][0][i]
        output[0][0][i] = output[0][0][i] + B_np[0][0][i]
    print(output)
numpy_caculate()
def print_npdata(np_data: np.ndarray) :
    print(np_data)
    print_num = 20
    d = np_data.flatten()
    p_size = print_num if d.size > print_num else d.size
    print(d[:p_size])

cuda -keys=cuda,gpu -arch=sm_61 -max_num_threads=1024 -max_shared_memory_per_block=49152 -max_threads_per_block=1024 -registers_per_block=65536 -thread_warp_size=32
[[[-20000. -19104. -19776. -18672. -20320. -18688. -18784. -19120.
   -17808. -18256.]]]
[[[-20000. -19104. -19776. -18672. -20320. -18688. -18784. -19120.
   -17808. -18256.]]]


In [53]:
# cuda未优化版本测试
sch = tvm.tir.Schedule(ModuleSrc)
with target:
    src_gpu_mod = tvm.tir.transform.DefaultGPUSchedule()(sch.mod) ##
rt_mod = tvm.build(src_gpu_mod, target="cuda")
W_w_nd = tvm.nd.array(W_w_np, dev)
W_s_nd = tvm.nd.array(W_s_np, dev)
B_nd = tvm.nd.array(B_np, dev)
Input_nd = tvm.nd.array(Input_np, dev)
Output_nd = tvm.nd.array(np.zeros((1, 1, w_y), dtype="float16"), dev)
evaluator = rt_mod.time_evaluator("main", dev, number=32)
print("manual_evaluator GEMV-Blocking: %f GFLOPS" % (num_flop / evaluator(W_w_nd, W_s_nd, Input_nd, B_nd, Output_nd).mean / 1e9))
# print(Output_nd.numpy())
print_npdata(Output_nd.numpy())

manual_evaluator GEMV-Blocking: 126.260829 GFLOPS
[[[-19360. -18912. -18976. ... -19408. -17728. -18272.]]]
[-19360. -18912. -18976. -19840. -19008. -19360. -18864. -18656. -19392.
 -19232. -19120. -19056. -19680. -18816. -18512. -18848. -18416. -20048.
 -19984. -18592.]


In [56]:
# cuda优化版本测试
rt_mod = tvm.build(sch_manual.mod, target="cuda")
W_w_nd = tvm.nd.array(W_w_np, dev)
W_s_nd = tvm.nd.array(W_s_np, dev)
B_nd = tvm.nd.array(B_np, dev)
Input_nd = tvm.nd.array(Input_np, dev)
Output_nd = tvm.nd.array(np.zeros((1, 1, w_y), dtype="float16"), dev)
evaluator = rt_mod.time_evaluator("main", dev, number=32)
print("manual_evaluator GEMV-Blocking: %f GFLOPS" % (num_flop / evaluator(W_w_nd, W_s_nd, Input_nd, B_nd, Output_nd).mean / 1e9))
# print(Output_nd.numpy())
print_npdata(Output_nd.numpy())

manual_evaluator GEMV-Blocking: 199.075418 GFLOPS
[[[-20000. -19136. -19776. ... -18976. -18240. -18496.]]]
[-20000. -19136. -19776. -18672. -20320. -18688. -18768. -19120. -17808.
 -18256. -18416. -20128. -19120. -18912. -19024. -19008. -18928. -18032.
 -18944. -19664.]


In [45]:
os.environ["TVM_NDK_CC"]="/home/sensetime/Android/Sdk/ndk/25.2.9519653/toolchains/llvm/prebuilt/linux-x86_64/bin/aarch64-linux-android33-clang++"
target = tvm.target.Target("opencl -device=adreno", host="llvm -mtriple=aarch64-linux-gnu")
device_key="android"
rpc_host = "10.4.236.32"
rpc_port = 9190
comp_target = tvm.target.Target("opencl", host="llvm -mtriple=aarch64-linux-android")  # TODO: Only support arm64 for now

def test_opencl(mod: tvm.IRModule, name_hint: str):
    # mod = tvm.lower(sch_manual.mod)
    print("Build ...")
    android_rt_mod = tvm.build(mod, target="opencl", target_host="llvm -mtriple=aarch64-linux-android")
    # print(android_rt_mod.imported_modules[0].get_source())
    temp = utils.tempdir()
    path_dso_cl = temp.relpath("dev_lib_cl.so")
    android_rt_mod.export_library(path_dso_cl, ndk.create_shared)

    print("Run GPU(OpenCL Flavor) test ...")
    # Establish remote connection with target hardware

    tracker = rpc.connect_tracker(rpc_host, rpc_port)
    remote = tracker.request(device_key, priority=0, session_timeout=60)
    print("Connect to device done.")
    dev = remote.cl(0)
    remote.upload(path_dso_cl)
    f1 = remote.load_module("dev_lib_cl.so")

    W_w_nd = tvm.nd.array(W_w_np, dev)
    W_s_nd = tvm.nd.array(W_s_np, dev)
    B_nd = tvm.nd.array(B_np, dev)
    Input_nd = tvm.nd.array(Input_np, dev)
    Output_nd = tvm.nd.array(np.zeros((1, 1, w_y), dtype="float16"), dev)
    test_number=32
    time_f = f1.time_evaluator(f1.entry_name, dev, number=test_number)
    cost = time_f(W_w_nd, W_s_nd, Input_nd, B_nd, Output_nd).mean
    print("evaluator[%s] GEMV-Blocking: %fms with loop %d" % (name_hint, cost * 1000, test_number))
    print("evaluator[%s] GEMV-Blocking: %fGFLOPS" % (name_hint, num_flop / cost / 1e9))
    print_npdata(Output_nd.numpy())
    # return Output_nd.numpy()
    return cost * 1000

In [57]:
# 未优化版本opencl测试
from tvm import dlight as dl
sch = tvm.tir.Schedule(ModuleSrc)
with target:
    # src_gpu_mod = tvm.tir.transform.DefaultGPUSchedule()(sch.mod) ##
    mod_deploy = dl.ApplyDefaultSchedule(  # pylint: disable=not-callable
        dl.gpu.Matmul(),
        dl.gpu.GEMV(),
        dl.gpu.Reduction(),
        dl.gpu.GeneralReduction(),
        dl.gpu.Fallback(),
    )(sch.mod)
src_output = test_opencl(mod_deploy, "source")
print_npdata(src_output)


Build ...
Run GPU(OpenCL Flavor) test ...


/data/workspace/llm/github/new_wksp/tvm-unity/python/tvm/driver/build_module.py:264: UserWarning: target_host parameter is going to be deprecated. Please pass in tvm.target.Target(target, host=target_host) instead.
  warnings.warn(


Connect to device done.
evaluator[source] GEMV-Blocking: 2.098032ms with loop 32
evaluator[source] GEMV-Blocking: 585.504313GFLOPS
[[[-20176. -19312. -19920. ... -19136. -18432. -18688.]]]
[-20176. -19312. -19920. -18944. -20512. -18912. -18928. -19344. -18016.
 -18432. -18576. -20272. -19328. -19136. -19232. -19232. -19136. -18160.
 -19056. -19872.]
2.098032
[2.098032]


In [58]:
#优化版本opencl测试
# print(sch_manual.mod)
opt_output = test_opencl(sch_manual.mod, "opted")
# print_npdata(opt_output)
# np.testing.assert_equal(opt_output, src_output)

Build ...
Run GPU(OpenCL Flavor) test ...
Connect to device done.
evaluator[opted] GEMV-Blocking: 0.717656ms with loop 32
evaluator[opted] GEMV-Blocking: 1711.693045GFLOPS
[[[-20000. -19136. -19776. ... -18976. -18240. -18496.]]]
[-20000. -19136. -19776. -18672. -20336. -18688. -18784. -19120. -17808.
 -18256. -18432. -20128. -19136. -18912. -19024. -19008. -18928. -18032.
 -18944. -19680.]


In [63]:
# 自动搜索
def auto_tune(record_file: str):
    from typing import Union
    def search(vectorize_factor: int, vectorize_input: int, threadIdxX: int, threadIdxY: int, blockIdxX: Union[int, None] = None):
        """search by workgroup

        Args:
            blockIdxX (_type_): blockIdx.x
            threadIdxX (_type_): threadIdx.x
            vectorize_output (_type_): 输出的vectorize参数, 决定单线程输出多少个结果
            vectorize_input (list, optional): 输入X拷贝到shared_memory时的vectorize参数, 一般为4或8

        Returns:
            _type_: _description_
        """
        @I.ir_module
        class ModuleToManual:
            @T.prim_func(private=False)
            def main(lv575: T.Buffer((T.int64(1376), T.int64(4096)), "uint32"), lv576: T.Buffer((T.int64(344), T.int64(4096)), "float16"), lv574: T.Buffer((T.int64(1), T.int64(1), T.int64(11008)), "float16"), lv570: T.Buffer((T.int64(1), T.int64(1), T.int64(4096)), "float16"), p_output0_intermediate: T.Buffer((T.int64(1), T.int64(1), T.int64(4096)), "float16")):
                T.func_attr({"global_symbol": "main", "tir.noalias": T.bool(True)})
                # with T.block("root"):
                p_output0_intermediate_1 = T.alloc_buffer((T.int64(11008), T.int64(4096)), "float16")
                var_matmul_intermediate = T.alloc_buffer((T.int64(1), T.int64(1), T.int64(4096)), "float16")
                for i, j in T.grid(T.int64(11008), T.int64(4096)):
                    with T.block("decode"):
                        v_i, v_j = T.axis.remap("SS", [i, j])
                        T.reads(lv575[v_i // T.int64(8), v_j], lv576[v_i // T.int64(32), v_j])
                        T.writes(p_output0_intermediate_1[v_i, v_j])
                        p_output0_intermediate_1[v_i, v_j] = (T.Cast("float16", T.bitwise_and(T.shift_right(lv575[v_i // T.int64(8), v_j], T.Cast("uint32", v_i % T.int64(8)) * T.uint32(4)), T.uint32(15))) - T.float16(7)) * lv576[v_i // T.int64(32), v_j]
                for i0, i1, i2, k in T.grid(T.int64(1), T.int64(1), T.int64(4096), T.int64(11008)):
                    with T.block("matmul"):
                        v_i0, v_i1, v_i2, v_k = T.axis.remap("SSSR", [i0, i1, i2, k])
                        T.reads(lv574[v_i0, v_i1, v_k], p_output0_intermediate_1[v_k, v_i2])
                        T.writes(var_matmul_intermediate[v_i0, v_i1, v_i2])
                        with T.init():
                            var_matmul_intermediate[v_i0, v_i1, v_i2] = T.float16(0)
                        var_matmul_intermediate[v_i0, v_i1, v_i2] = var_matmul_intermediate[v_i0, v_i1, v_i2] + lv574[v_i0, v_i1, v_k] * p_output0_intermediate_1[v_k, v_i2]
                for ax0, ax1, ax2 in T.grid(T.int64(1), T.int64(1), T.int64(4096)):
                    with T.block("T_add"):
                        v_ax0, v_ax1, v_ax2 = T.axis.remap("SSS", [ax0, ax1, ax2])
                        T.reads(lv570[v_ax0, v_ax1, v_ax2], var_matmul_intermediate[v_ax0, v_ax1, v_ax2])
                        T.writes(p_output0_intermediate[v_ax0, v_ax1, v_ax2])
                        p_output0_intermediate[v_ax0, v_ax1, v_ax2] = lv570[v_ax0, v_ax1, v_ax2] + var_matmul_intermediate[v_ax0, v_ax1, v_ax2]

        sch = tvm.tir.Schedule(ModuleToManual)
        b0 = sch.get_block(name="decode", func_name="main")
        b1 = sch.get_block(name="matmul", func_name="main")
        # 1 1 4096 11008
        l2, l3, l4, l5 = sch.get_loops(block=b1)
        # l6 = 4096
        l6 = sch.fuse(l2, l3, l4, preserve_unit_iters=True)
        v7, v8, v9 = sch.sample_perfect_tile(
            # loop=l6, n=3, max_innermost_factor=4, decision=[8, 256, 2]
            loop=l6, n=3, max_innermost_factor=4, decision=[blockIdxX, threadIdxX, vectorize_factor]
        )
        # 8  128   4
        l10, l11, l12 = sch.split(loop=l6, factors=[v7, v8, v9], preserve_unit_iters=True)
        v13, v14, v15 = sch.sample_perfect_tile(
            loop=l5, n=3, max_innermost_factor=8, decision=[344, 4, 8]
        )
        l16, l17, l18 = sch.split(
            loop=l5, factors=[v13, v14, v15], preserve_unit_iters=True
        )
        ## 搜索记录:
        #       2, 172  0.939960ms
        #       4, 86   0.757256ms
        #       8, 43   716400ms
        # v161, v162 = sch.sample_perfect_tile(loop=l16, n=2, max_innermost_factor=172, decision=[4, 86])
        l161, _ = sch.split(l16, [threadIdxY, 344//threadIdxY], preserve_unit_iters=True)
        # 将reduce操作按照l161做拆分，以便并行化. 其过程是: 
        #   1. 申请 [l161, 1, 1, 4096]个xxx_rf内存
        #   2. 将计算拆为l161个chunks分别计算
        #   3. 按照l161维度reduce, 得到最终的输出
        sch.rfactor(l161, 0)
        ## 此时b1还是block('matmul'), 但是计算内容变为rf的合并操作(即: reduce = sum(sub_reduces))
        ## 此时拆分的reduce计算变为 block('matmul_rf')
        b2 = sch.get_block(name="matmul_rf", func_name = "main")
        # 调用rfactor后, 计算部分转移到了 block('matmul_rf'), 所以得重新获取loops
        # s = spatial,  r = reduce
        lb2s1, lb2s2, lb2s3, lb2r1, lb2r2, lb2r3, lb2r4 = sch.get_loops(block=b2)
        # 8, 128, 8, 43, 4, 8, 4
        sch.reorder(lb2s1, lb2s2, lb2r1, lb2r2, lb2r3, lb2r4, lb2s3)
        sch.bind(loop=lb2s1, thread_axis="blockIdx.x")
        sch.bind(loop=lb2s2, thread_axis="threadIdx.x")
        sch.bind(loop=lb2r1, thread_axis="threadIdx.y")

        sch.compute_inline(block=b0)
        b21 = sch.cache_write(block = b2, write_buffer_index=0, storage_scope="local")
        sch.reverse_compute_at(block = b21, loop=lb2r1, preserve_unit_loops=True, index=-1)
        lb21, lb22, lb23, lb24, lb25, lb26, lb27 = sch.get_loops(block=b2)
        sch.vectorize(lb27)
        lb211, lb212, lb213, lb214, lb215,lb216, lb217 = sch.get_loops(block=b21)
        sch.vectorize(lb217)

        # 权重W uint32: [1376, 4096]
        b80 = sch.cache_read(block=b2, read_buffer_index=1, storage_scope="local")
        sch.compute_at(block=b80, loop=lb2r3, preserve_unit_loops=True, index=-1)
        l801, l802, l803, l804, l805, l806, l807 = sch.get_loops(block=b80)
        sch.vectorize(l807)

        # 权重Scale fp16: [344, 4096]
        b81 = sch.cache_read(block=b2, read_buffer_index=2, storage_scope="local")
        sch.compute_at(block=b81, loop=lb2r2, preserve_unit_loops=True, index=-1)
        l811, l812, l813, l814, l815, l816 = sch.get_loops(block=b81)
        sch.vectorize(l816)

        # 输入X: lv574_shared, shape[1,1,11008] -> [8, 128, 8, 1, 1, 11008/(8*128*8)] (无法整除)-> [8, 128, 8, 1, 1, 11008]
        b20 = sch.cache_read(block=b2, read_buffer_index=0, storage_scope="shared")
        sch.compute_at(block=b20, loop=lb2r1, preserve_unit_loops=True, index=-1)

        lb20l1, lb20l2, lb20l3, lb20l4, lb20l5, lb20l6 = sch.get_loops(block=b20)
        l34, l35, l36, l37 = sch.split(
            # loop=l33, factors=[None, 256, 8], preserve_unit_iters=True
            ### 注意这里的数字与最开始设置保持一致
            # loop=lb20l6, factors=[None, 256, 4, 4], preserve_unit_iters=True
            loop=lb20l6, factors=[None, threadIdxX, threadIdxY, vectorize_input], preserve_unit_iters=True
        )
        # 这里 l37 = 4, 11008 / (threadIdx.x * threadIdx.y * 4) 无法整除, 所以只有一部分线程可以执行到shared_memory构建
        # 实际观测 sch.vectorize(loop=l37) 并没有生效，不知道是否因为不是所有线程都能执行, 所以取消了vload4
        sch.vectorize(loop=l37)
        sch.bind(loop=l35, thread_axis="threadIdx.x")
        sch.bind(loop=l36, thread_axis="threadIdx.y")

        v21 = sch.sample_categorical(
            candidates=[1, 2, 4, 8], probs=[0.25, 0.25, 0.25, 0.25], decision=[1, 2, 4, 8].index(vectorize_input)
        )
        sch.annotate(
            block_or_loop=b20, ann_key="meta_schedule.cooperative_fetch", ann_val=v21
        )
        sch.vectorize(loop=l12)
        b27 = sch.decompose_reduction(block=b2, loop=lb2r1)
        sch.enter_postproc()
        sch.unannotate(block_or_loop=b20, ann_key="meta_schedule.cooperative_fetch")
        
        #######################################################################################
        ###################以下为kenenl1: 执行 sub_reduce结果合并 & bias操作#######################
        #######################################################################################
        # 计算的中间结果: var_matmul_intermediate_rf, shape[8, 1,1,4096]
        #  8   128   4
        # l10, l11, l12
        b19 = sch.cache_write(block=b1, write_buffer_index=0, storage_scope="local")
        l41,l42,l43,l44 = sch.get_loops(block=b1)
        l411 = sch.fuse(l41,l42, l43)
        ## 调整记录:
        #       4, 256, 4 0.717400ms
        #       8, 128, 4 0.717368ms
        #       16, 64, 4 0.717368ms
        #       32, 32, 4 0.716808ms
        #       64, 16, 4 0.718056ms

        #       4, 128, 8 0.719936ms
        #       8, 64,  8 0.719024ms
        #       16, 32, 8 0.716400ms best
        #       32, 16, 8 0.717544ms

        #       4, 64, 16 0.722680ms
        #       8, 32, 16 0.719608ms
        #       16, 16, 16 0.718744ms
        #       32, 8, 16 0.720376ms
        l411_1, l411_2, l411_3 = sch.split(l411, [16, 32, 8], preserve_unit_iters=True)
        sch.reverse_compute_at(block=b19, loop=l411_2, preserve_unit_loops=True, index=-1)
        b28 = sch.get_block(name="T_add", func_name="main")
        sch.reverse_compute_inline(block=b28)
        l22, l23, l24, l25, l26 = sch.get_loops(block=b19)
        l41,l42,l43,l44 = sch.get_loops(block=b1)
        sch.vectorize(loop=l43)
        sch.bind(loop=l41, thread_axis="blockIdx.x")
        sch.bind(loop=l42, thread_axis="threadIdx.x")
        return sch.mod

    vec_factor = [2, 4, 8]
    vec_input = [2, 4, 8]
    blockx = [None]
    threadx = [16, 32, 64, 128, 256, 512]
    thready = [2, 4, 8]
    task_index = 0
    total_task_num = len(vec_factor)*len(vec_input)*len(blockx)*len(threadx) * len(thready)
    records = {}
    print(f"Total tasks: {total_task_num}")
    write_interval = 5
    from prettytable import PrettyTable
    table = PrettyTable()
    table.field_names = ["vf", "vi", "tx", "ty", "bx", "cost(ms)"]
    for vf in vec_factor:
        for vi in vec_input:
            for tx in threadx:
                for ty in thready:
                    task_index = task_index + 1
                    import math
                    bx = math.ceil(w_y /(vf * tx))
                    if bx * tx * vf > w_y or tx * ty > 1024 or 344 % ty != 0  or w_y % (vf * tx) != 0:
                        print(f"search record [{task_index}/{total_task_num}]: skip {vf} {vi} {tx} {ty} {bx}")
                        continue
                    print(f"search record [{task_index}/{total_task_num}]: start run {vf} {vi} {tx} {ty} {bx}")
                    mod_deploy = search(vf, vi, tx, ty, bx)
                    cost = test_opencl(mod_deploy, "search")
                    print("=====")
                    records[(vf, vi, tx, ty, bx)] = cost
                    table.add_row([vf, vi, tx, ty, bx, cost])
                    if task_index % write_interval == 0:
                        with open(record_file, 'wt') as f:
                            f.write(table.get_csv_string())
    print("================================")
    print(table)
    
    # record_sorted = sorted(record.items(), key=lambda x: x[1][0], reverse=True)
auto_tune("./mlp_down_tune_record_1.csv")

Total tasks: 162
search record [1/162]: start run 2 2 16 2 128
Build ...


/data/workspace/llm/github/new_wksp/tvm-unity/python/tvm/driver/build_module.py:264: UserWarning: target_host parameter is going to be deprecated. Please pass in tvm.target.Target(target, host=target_host) instead.
  warnings.warn(


Run GPU(OpenCL Flavor) test ...
Connect to device done.
evaluator[search] GEMV-Blocking: 7.009120ms with loop 32
evaluator[search] GEMV-Blocking: 175.258347GFLOPS
[[[-17616. -17040. -17248. ... -16928. -16560. -16288.]]]
[-17616. -17040. -17248. -16672. -18000. -16784. -16960. -16944. -16160.
 -16312. -16352. -17648. -17056. -16800. -16736. -16864. -16864. -16192.
 -16720. -17120.]
=====
search record [2/162]: start run 2 2 16 4 128
Build ...
Run GPU(OpenCL Flavor) test ...
Connect to device done.
evaluator[search] GEMV-Blocking: 3.906928ms with loop 32
evaluator[search] GEMV-Blocking: 314.417564GFLOPS
[[[-19328. -18496. -19200. ... -18336. -17712. -17984.]]]
[-19328. -18496. -19200. -18096. -19680. -18064. -18176. -18608. -17312.
 -17696. -17824. -19456. -18512. -18208. -18432. -18416. -18448. -17520.
 -18304. -19152.]
=====
search record [3/162]: start run 2 2 16 8 128
Build ...
Run GPU(OpenCL Flavor) test ...
Connect to device done.
evaluator[search] GEMV-Blocking: 2.436456ms with l

In [ ]:
import numpy as np
target = tvm.target.Target("opencl -device=adreno", host="llvm -mtriple=aarch64-linux-gnu")
device_key="android"
rpc_host = "10.158.176.30"
rpc_port = 5001
# remote = autotvm.measure.request_remote(device_key, "10.158.176.30", 5001, timeout=10000)
# dev = remote.device(str(target), 0)

# num_flop = 1228406784
# W_np = np.random.uniform(size=(512, vocab_size)).astype("uint32")
# S_np = np.random.uniform(size=(128, vocab_size)).astype("float16")
# Input_np = np.random.uniform(size=(1, 1, 4096)).astype("float16")
# # Output_np = np.random.uniform(size=(1, 1, 4096)).astype("float16")
# W_nd = tvm.nd.array(W_np, dev)
# S_nd = tvm.nd.array(S_np, dev)
# Input_nd = tvm.nd.array(Input_np, dev)
# Output_nd = tvm.nd.array(np.zeros((1, 1, vocab_size), dtype="float32"), dev)

In [ ]:
rpc_config = ms.runner.RPCConfig(tracker_host=rpc_host, tracker_port=rpc_port, tracker_key = device_key)
runner= ms.runner.RPCRunner(rpc_config)
# ms.builder.LocalBuilder()
sch = tvm.tir.Schedule(ModuleSrc)
database = ms.tune_tir(
    mod=ModuleSrc,
    target=target,
    max_trials_global=64,
    num_trials_per_iter=64,
    work_dir="./tune_first",
    cost_model="xgb",
    runner = runner
)
print(len(database))
sch1 = ms.tir_integration.compile_tir(database, sch.mod, target)
print(type(sch1))

In [ ]:
from tvm.script import relax as R
@I.ir_module
class Module:
    @R.function
    def main(A: R.Tensor((3, 4), dtype="float32"), B: R.Tensor((4, 5), dtype="float32")):
        with R.dataflow():
            lv: R.Tensor((3, 5), dtype="float32") = R.matmul(A, B)
            gv: R.Tensor((3, 5), dtype="float32") = lv
            R.output(gv)
        return gv

In [ ]:
## auto_scheduler test
from tvm import auto_scheduler
import numpy as np
a_np = np.random.rand(3, 4).astype("float32")
b_np = np.random.rand(4, 5).astype("float32")
a_nd = tvm.runtime.NDArray(a_np)
b_nd = tvm.runtime.NDArray(b_np)
sch = tvm.tir.Schedule(Module)

params = {"A": a_np, "B": b_np}
## 报错，这里只支持relay
# tasks = auto_scheduler.extract_tasks(sch.mod, params, target=target)
tasks = ms.relax_integration.extract_tasks(sch.mod, target=target, params=params)
print(len(tasks))

In [ ]:

from mod_deploy import Module as ModuleAll
params_all = {}
tasks_all = auto_scheduler.extract_tasks(ModuleAll, params_all, target=target)
print(len(tasks_all))

In [ ]:
import numpy as np
log_file = "tune.json"
def _detect_local_cuda():
    dev = tvm.cuda()
    if not dev.exist:
        return None
    return tvm.target.Target(
        {
            "kind": "cuda",
            "max_shared_memory_per_block": dev.max_shared_memory_per_block,
            "max_threads_per_block": dev.max_threads_per_block,
            "thread_warp_size": dev.warp_size,
            "registers_per_block": 65536,
            "arch": "sm_" + tvm.cuda().compute_version.replace(".", ""),
        }
    )
# target = tvm.target.Target("cuda", host="llvm")
target = _detect_local_cuda()

print(target)
# 定义计算任务
dev = tvm.cuda(0)

num_flop = 1228406784
W_np = np.random.uniform(size=(512, vocab_size)).astype("uint32")
S_np = np.random.uniform(size=(128, vocab_size)).astype("float16")
Input_np = np.random.uniform(size=(1, 1, 4096)).astype("float16")
# Output_np = np.random.uniform(size=(1, 1, 4096)).astype("float16")
W_nd = tvm.nd.array(W_np, dev)
S_nd = tvm.nd.array(S_np, dev)
Input_nd = tvm.nd.array(Input_np, dev)
Output_nd = tvm.nd.array(np.zeros((1, 1, vocab_size), dtype="float32"), dev)
sch = tvm.tir.Schedule(ModuleSrc)
new_mod = sch.mod


In [ ]:
# task = auto_scheduler.SearchTask(func=sch.mod['fused_fused_decode11_fused_matmul5_cast2'], args=sch.mod['fused_fused_decode11_fused_matmul5_cast2'].params, target=target)

# tune_option = auto_scheduler.TuningOptions(
#     num_measure_trials=10,
#     measure_callbacks=[auto_scheduler.RecordToFile(log_file)],
#     verbose=2,
# )


database = ms.tune_tir(
    mod=new_mod,
    target=target,
    max_trials_global=64,
    num_trials_per_iter=64,
    work_dir="./tune_45593_1",
    cost_model="xgb"
)
print(len(database))
sch1 = ms.tir_integration.compile_tir(database, new_mod, target)
print(type(sch1))

In [ ]:
# print(sch1.trace)
# print(sch1.mod.script())
rt_mod = tvm.build(sch1.mod, target="cuda")

evaluator = rt_mod.time_evaluator("main", dev, number=100)

print("evaluator GEMV-Blocking: %f GFLOPS" % (1228406784 / evaluator(W_nd, S_nd, Input_nd, Output_nd).mean / 1e9))




In [ ]:

record_database = ms.Database.create(kind='json', work_dir='./tune_45593_1')


In [ ]:
record_sch = ms.tir_integration.compile_tir(record_database, new_mod, target)

record_rt_mod = tvm.build(record_sch.mod, target="cuda")

record_evaluator = record_rt_mod.time_evaluator("main", dev, number=20)

print("evaluator GEMV-Blocking: %f GFLOPS" % (num_flop / record_evaluator(W_nd, S_nd, Input_nd, Output_nd).mean / 1e9))
print(record_sch.trace)
print(record_sch.mod.script())

In [ ]:
from typing import TYPE_CHECKING, Dict, List, Optional, Union, Callable
from tvm import runtime
if TYPE_CHECKING:
    import numpy as np  # type: ignore
    from tvm.ir import IRModule
    from tvm.meta_schedule.runner import EvaluatorConfig, RPCConfig
    from tvm.runtime import Device, Module, NDArray
    from tvm.target import Target
    from tvm.runtime.vm import Executable


def f_measurement(
    rt_mod: runtime.Module, device: runtime.ndarray.Device, input_data: Dict[str, runtime.NDArray]
):
    vm = relax.VirtualMachine(rt_mod, device=device)
    vm.save_function("main", "measure_func", **input_data, include_return=False)
    evaluator = vm.time_evaluator(
        func_name="measure_func",
        dev=device,
        repeat=100,
        number=1,
        min_repeat_ms=500,
    )
    return evaluator()

def run_module_via_rpc(
    rpc_config: "RPCConfig",
    lib: Union["Module", "Executable"],
    dev_type: str,
    args: Union[Dict[int, "np.ndarray"], Dict[str, "np.ndarray"]],
    continuation: Callable,
    backend: Optional[str] = "graph",
):
    """Execute a tvm.runtime.Module on RPC remote"""
    # pylint: disable=import-outside-toplevel
    import os
    import tempfile

    from tvm.contrib.tar import tar
    from tvm.runtime import ndarray

    # pylint: enable=import-outside-toplevel

    with tempfile.TemporaryDirectory() as tmp_dir:
        # filename = os.path.join(tmp_dir, "tvm_tmp_mod." + tar.output_format)
        filename = os.path.join(tmp_dir, "tvm_tmp_mod." + "so")
        if backend == "vm":
            code, lib = lib.save(filename, fmt="so")
        from tvm.contrib import ndk
        lib.export_library(filename, ndk.create_shared)
        session = rpc_config.connect_server()
        print(type(session._sess))
        session.upload(filename)
        _, filename = os.path.split(filename)
        rt_mod = session.load_module(filename)
        
        if backend == "vm":
            rt_mod = session.get_function("runtime.Load_Executable")(code, rt_mod)
            # rt_mod = session.get_function("runtime.module.loadfile_relax.Executable")(filename)
        dev = session.device(dev_type=dev_type, dev_id=0)
        # print(dev)
        # create the remote runtime module
        print(rt_mod)
        print(rt_mod['main'])
        from tvm.contrib import graph_executor as runtime
        module = runtime.GraphModule(rt_mod["main"](dev))
        print(module)
        for k, v in args.items():
            module.set_input(k, tvm.nd.array(v))
        return module.run()
        # nd_args = {k: ndarray.array(v, dev) for k, v in args.items()}
        nd_args = {k: ndarray.empty(v.shape, v.dtype, dev) for k, v in args.items()}
        return continuation(rt_mod, dev, nd_args)